In [26]:
import numpy as np
import cv2
import os

In [27]:
def downsampling_image(image, reduction_scale) : # Reduce image resolution Pyr lelel
    for i in range(0, reduction_scale) :
        if len(image.shape) > 2:
            row, col = image.shape[:2]
        else :
            row, col = image.shape
        
        image = cv2.pyrDown(image,dstsize=(col//2,row//2))
    return imag

In [28]:
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

In [29]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [30]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.65*n.distance : #
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 10.5) # H RANSAC 
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (100,100,200), 5, cv2.LINE_AA)# ความหนาเส้น สี
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return query_img,query_img

In [31]:
vid = cv2.VideoCapture('test/videos/final_exam/Dataset-1/right_output-1.avi')

# เปิดภาพ
template_img = cv2.imread('images/final_exam/Templates/Template-2.png')
# ทำรูปขาวดำ
template_gray =  cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)

while vid.isOpened() :
    ret, frame = vid.read()
    
    if ret :
        #แปลงวิดีโอให้เป็นขาวดำ
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 

        #จำนวนการจับคู่ ห้ามน้อยกว่า 10
        detected, drawmatch =  feature_object_detection(template_img, template_gray, frame, frame_gray, 12)

        cv2.imshow('Video frame', detected)

        if cv2.waitKey(int(1000/1000)) & 0xFF == ord('q') : # this line control the period between image frame 
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()

Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
Keypoints not enough
